#### To Do
- Don't upload if already exists in bucket folder
- Make unzip_blob a subflow?
- Delete zip blob, once unzip complete
- Use pyspark or dbt to get into BQ?

In [81]:
from io import BytesIO
from pathlib import Path
from zipfile import ZipFile, is_zipfile
import pandas as pd
from prefect import flow, task
from google.cloud import storage
import os

In [82]:
# catch = pd.read_csv('../tmp/catch_19814.csv')
# size = pd.read_csv('../tmp/size_19826.csv')
# trip = pd.read_csv('../tmp/trip_19842.csv')

# catch_dtype = catch.dtypes.to_dict()
# size_dtype = size.dtypes.to_dict()
# trip_dtype = trip.dtypes.to_dict()

# print(size_dtype)

In [83]:
catch_dtype = {'common': str, 'strat_id': str, 'psu_id': int, 
                'id_code': int, 'MODE_FX': int, 'AREA_X': int, 
                'ST': int, 'SUB_REG': int, 'WAVE': int, 
                'year': int, 'month': int, 'kod': str, 
                'SP_CODE': float, 'CLAIM': float, 'RELEASE': float, 
                'HARVEST': float, 'CLAIM_UNADJ': int, 'HARVEST_UNADJ': int, 
                'RELEASE_UNADJ': int, 'tot_len_a': float, 'wgt_a': float, 
                'tot_len_b1': float, 'wgt_b1': float, 'tot_cat': float, 
                'wgt_ab1': float, 'tot_len': float, 'Landing': float, 
                'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
                'fl_reg': int, 'wp_catch_precal': float, 'wp_int': float, 
                'wp_catch': float, 'date_published': str}

trip_dtype = {'prim2_common': str, 'prim1_common': str, 'strat_id': str, 
              'psu_id': int, 'ADD_HRS': float, 'AREA': int, 
              'AREA_X': int, 'BOAT_HRS': float, 'CATCH': int, 
              'CNTRBTRS': int, 'CNTY': int, 'CNTY_RES': int, 
              'COASTAL': str, 'FFDAYS2': int, 'FFDAYS12': int, 
              'FIRST': float, 'HRSF': float, 'ID_CODE': int, 
              'INTSITE': int, 'MODE_F': int, 'MODE_FX': int, 
              'NUM_TYP2': int, 'NUM_TYP3': int, 'NUM_TYP4': int, 
              'NUM_TYP6': float, 'ON_LIST': float, 'PARTY': float, 
              'REG_RES': float, 'ST': int, 'ST_RES': int, 
              'SUB_REG': int, 'TELEFON': int, 'WAVE': int, 
              'YEAR': int, 'ASG_CODE': int, 'month': int, 
              'kod': str, 'MODE_ASG': float, 'new_list': float, 
              'PRT_CODE': int, 'CELLTYPE': float, 'fshinsp_a': float, 
              'num_fish_a': float, 'fl_reg': int, 'ADD_PH': int, 
              'AREA_NC': float, 'COUNTY': str, 'DATE1': str, 
              'DIST': int, 'F_BY_P': int, 'GEAR': int, 
              'MODE2001': float, 'MUNI_RES': float, 'MUNI_TRP': float, 
              'PRIM1': float, 'PRIM2': float, 'PVT_RES': int, 
              'RIG': int, 'SEP_FISH': int, 'TIME': int, 
              'ZIP': float, 'AGE': float, 'wp_int': float, 
              'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
              'LEADER': int, 'date_published': str}

size_dtype = {'AREA_X': int, 'ID_CODE': int, 'MODE_FX': int, 
              'ST': int, 'SUB_REG': int, 'WAVE': int, 
              'YEAR': int, 'month': int, 'kod': str, 
              'SP_CODE': float, 'LNGTH': float, 'WGT': float, 
              'lngth_imp': int, 'wgt_imp': int, 'strat_id': str, 
              'psu_id': int, 'common': str, 'wgt_unadj': float, 
              'wp_size': float, 'l_in_bin': float, 'l_cm_bin': float, 
              'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
              'date_published': str}

In [84]:
@task()
def read_gcs_bucket(bucket) -> list:
    """Get list of zip folders in GCS bucket"""
    lsblob = list(bucket.list_blobs(prefix="zip"))  # get list of blobs in zip folder of bucket
    lsblob = [l.name for l in lsblob]  # only return the filename from the blobs
    print(lsblob)

    return lsblob

c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\tasks.py:275: UserWarning: A task named 'read_gcs_bucket' and defined at 'C:\Users\jlopez\AppData\Local\Temp\ipykernel_13524\561035817.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [85]:
@task()
def process_file(filename: str, file: str, bucket) -> pd.DataFrame:
    """Convert csv file to parquet file
    (csv) -> parquet"""
    tmp_dir = "../tmp"
    Path(tmp_dir).mkdir(parents=True, exist_ok=True)

    folder = filename.split("_")[0]
    trunc_fn = filename.split(".")[0]
    output_file = f"../tmp/{trunc_fn}.csv"
    outfile = open(output_file, "wb")
    outfile.write(file)
    outfile.close()

    file_extension = 'csv'
    if folder == "catch":
        dtype = catch_dtype
        blob = bucket.blob(f"catch/{trunc_fn}.{file_extension}")
    elif folder == "trip":
        dtype = trip_dtype
        blob = bucket.blob(f"trip/{trunc_fn}.{file_extension}")
    elif folder == "size":
        dtype = size_dtype
        blob = bucket.blob(f"size/{trunc_fn}.{file_extension}")

    # df = pd.read_csv(output_file
    #                  #, dtype=dtype
    #                  , low_memory=False
    #                  )
    # df.to_parquet(output_file.replace("csv", "parquet"))

    return trunc_fn, output_file, blob

c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\tasks.py:275: UserWarning: A task named 'process_file' and defined at 'C:\Users\jlopez\AppData\Local\Temp\ipykernel_13524\2116000842.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [86]:
@task()
def write_gcs(filename: str, file, blob) -> None:
    """Write parquet file to GCS
    (Dataframe) -> None"""
    
    with open(file, "rb") as myfile:
        blob.upload_from_file(myfile)

    print(f'Csv file size is {os.stat(f"../tmp/{filename}.csv").st_size}')
    os.remove(f"../tmp/{filename}.csv")  # delete/remove outfile

    # print(f'Parquet file size is {os.stat(f"../tmp/{filename}.parquet").st_size}')
    # os.remove(f"../tmp/{filename}.parquet")  # delete/remove outfile

c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\tasks.py:275: UserWarning: A task named 'write_gcs' and defined at 'C:\Users\jlopez\AppData\Local\Temp\ipykernel_13524\1476261933.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [87]:
# def unzip_blob_csv(source_path: str, bucket) -> list:
#     """Unzip folder"""
#     blob = bucket.blob(source_path)

#     zipbytes = BytesIO(blob.download_as_string())

#     if is_zipfile(zipbytes):
#         with ZipFile(zipbytes, "r") as myzip:
#             for contentfilename in myzip.namelist():
#                 folder = contentfilename.split("_")[0]
#                 contentfile = myzip.read(contentfilename)

#                 tmp_dir = "../tmp"
#                 Path(tmp_dir).mkdir(parents=True, exist_ok=True)

#                 trunc_fn = contentfilename.split("_")[1]
#                 output_file = f"../tmp/{trunc_fn}"
#                 outfile = open(output_file, "wb")
#                 outfile.write(contentfile)
#                 outfile.close()

#                 if folder == "catch":
#                     blob = bucket.blob(f"catch/{trunc_fn}")
#                 elif folder == "trip":
#                     blob = bucket.blob(f"trip/{trunc_fn}")
#                 elif folder == "size":
#                     blob = bucket.blob(f"size/{trunc_fn}")

#                 with open(output_file, "rb") as mycsv:
#                     blob.upload_from_file(mycsv)

#                 os.remove(f"../tmp/{trunc_fn}")  # delete/remove outfile


In [88]:
@task()
def unzip_blob(source_path: str, bucket) -> dict:
    """Unzip folder"""
    blob = bucket.blob(source_path)

    zipbytes = BytesIO(blob.download_as_string())

    lsfilename = []
    lsfile = []

    if is_zipfile(zipbytes):
        with ZipFile(zipbytes, "r") as myzip:
            for contentfilename in myzip.namelist():
                contentfile = myzip.read(contentfilename)

                lsfilename.append(contentfilename)
                lsfile.append(contentfile)
    
    dictfile = dict(zip(lsfilename, lsfile))
    return dictfile

In [89]:
@flow(log_prints=False)
def process_gcs_blob(blob: str = "ps_2021_csv.zip"
                     , bucket_name: str = "de_project_bucket") -> None:
    """Process blob in gcs bucket"""
    storage_client = storage.Client.from_service_account_json("../creds.json")
    # bucket_name = "de_project_bucket"  # parameterize this
    bucket = storage_client.get_bucket(bucket_name)

    # unzip_blob(blob, bucket)
    csvs = unzip_blob(f'zip/{blob}', bucket)
    for k, v in csvs.items():
        print(k)
        pf = process_file(k, v, bucket)
        filename = pf[0] #get file name without extension
        file = pf[1]
        blob = pf[2]
        write_gcs(filename, file, blob)

    # lsblob = read_gcs_bucket(bucket)

    # for blob in lsblob:
    #     csvs = unzip_blob(blob, bucket)
    #     for k, v in csvs.items():
    #         print(k)
    #         parquet = process_file(k, v, bucket)
    #         filename = parquet[0] #get file name without extension
    #         file = parquet[1]
    #         blob = parquet[2]
    #         write_gcs(filename, file, blob)

c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\flows.py:214: UserWarning: A flow named 'process-gcs-blob' and defined at 'C:\Users\jlopez\AppData\Local\Temp\ipykernel_13524\747118278.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [90]:
if __name__ == '__main__':
    blob = "ps_1981_1989_csv.zip"
    bucket_name = "de_project_bucket"
    process_gcs_blob(blob, bucket_name)  

#57min with convert to parquet @ 3.63GB


22:26:44.278 | INFO    | prefect.engine - Created flow run 'khaki-bat' for flow 'process-gcs-blob'

22:26:44.988 | INFO    | Flow run 'khaki-bat' - Created task run 'unzip_blob-0' for task 'unzip_blob'

22:26:44.992 | INFO    | Flow run 'khaki-bat' - Executing 'unzip_blob-0' immediately...

22:26:49.124 | INFO    | Task run 'unzip_blob-0' - Finished in state Completed()

catch_19811.csv


...

22:34:03.068 | INFO    | Flow run 'khaki-bat' - Created task run 'write_gcs-161' for task 'write_gcs'

22:34:03.072 | INFO    | Flow run 'khaki-bat' - Executing 'write_gcs-161' immediately...

Csv file size is 1777045


22:34:04.826 | INFO    | Task run 'write_gcs-161' - Finished in state Completed()

22:34:05.161 | INFO    | Flow run 'khaki-bat' - Finished in state Completed('All states completed.')